In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchtext
import tqdm
from torchnlp.metrics import get_moses_multi_bleu
from torchtext.data import Field, BucketIterator
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

import tensorflow as tf
import tensorflow_datasets as tfds
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO

import linecache
import sys
import os
import re
import random
import time

In [2]:
!nvidia-smi

Sat Oct 26 12:45:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 430.40       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   34C    P8    14W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 27%   28C    P8    18W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

## Loading the dataset

In [3]:
try:
    os.mkdir("./datasets")
except FileExistsError:
    print("Directories already exists")

# getting descriptions
!wget https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.anno -O ./datasets/all.desc

# getting code
!wget https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.code -O ./datasets/all.code

Directories already exists
--2019-10-26 12:45:07--  https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.anno
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1382085 (1.3M) [text/plain]
Saving to: './datasets/all.desc'

./datasets/all.desc 100%[===================>]   1.32M  --.-KB/s    in 0.07s   

2019-10-26 12:45:07 (19.5 MB/s) - './datasets/all.desc' saved [1382085/1382085]

--2019-10-26 12:45:07--  https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.code
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 906732 (885K) [text/plain]
Saving to: './datasets/a

## Creating a token text encoder
An encoder will take a file and a splitting function and return an object able to encode and decode a string. It will also be able to save a vocab file and retrieve from file.

In [4]:
text = " append rel_to to string 'ForeignKey, (substitute the result for field_type.)"

# looks like code split need parenthesis to be matched in the same string, if not it gives an error...
def code_split(s):
    return [x.string for x in tokenize(BytesIO(s.encode('utf-8')).readline) if x.string != '' and x.string != "\n" and not x.string.isspace()][1:]

print(code_split(text))

['append', 'rel_to', 'to', 'string', "'", 'ForeignKey', ',', '(', 'substitute', 'the', 'result', 'for', 'field_type', '.', ')']


In [5]:
text = " append rel_to to string 'ForeignKey, (subs__titute the result' for field_type."

def string_split(s):
    return list(filter(lambda x: x != '' and x != "\n" and not x.isspace(), re.split('(_|\W)', s))) # this will chunk all code properly by plits strings with quotes
#     return list(filter(lambda x: x != '' and x != "\n" and not x.isspace(), re.split('(\\\'.*?\\\'|\\\".*?\\\"|_|\W)', s))) # this keeps the strings intact

print(string_split(text))

['append', 'rel', '_', 'to', 'to', 'string', "'", 'ForeignKey', ',', '(', 'subs', '_', '_', 'titute', 'the', 'result', "'", 'for', 'field', '_', 'type', '.']


## Making the input pipeline

In [6]:
def corpus_to_array(src_fp, tgt_fp):
    lines = []
    with open(src_fp, "r") as src_file, open(tgt_fp, "r") as tgt_file:
        for src, tgt in zip(src_file, tgt_file):
            lines.append((src, tgt))
    return lines

In [7]:
def filter_corpus(data, max_seq_length=200, tokenizer=string_split):
    return [(src, tgt) for src, tgt in data if len(string_split(src)) <= max_seq_length and len(string_split(tgt)) <= max_seq_length]

In [8]:
def samples_to_dataset(samples, src_field, tgt_field):
    """
    Args:
        samples: [(src_string),(tgt_string)]
        src/tgt_tokenizer: a func that takes a string and returns an array of strings
    """
    examples = []
    
    for sample in samples:
        src_string, tgt_string = sample
        examples.append(torchtext.data.Example.fromdict({"src":src_string, "tgt":tgt_string}, 
                                        fields={"src":("src",src_field), "tgt":("tgt",tgt_field)}))
        
    dataset = torchtext.data.Dataset(examples,fields={"src":src_field, "tgt":tgt_field})
    return dataset

In [9]:
data = corpus_to_array("datasets/all.desc", "datasets/all.code")
random.shuffle(data)

In [10]:
print("Max src length:", max([len(string_split(src)) for src, tgt in data]))
print("Max tgt length:", max([len(string_split(tgt)) for src, tgt in data]))

Max src length: 586
Max tgt length: 1087


In [11]:
print("Full dataset size:", len(data))
max_seq_length=200
data = filter_corpus(data, max_seq_length=200, tokenizer=string_split)
print("Limited dataset size:", len(data))

Full dataset size: 18805
Limited dataset size: 18781


In [12]:
SRC_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')
TGT_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')

dataset = samples_to_dataset(data, SRC_TEXT, TGT_TEXT)

train_dataset, val_dataset = dataset.split([0.7,0.3])

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0) # choose GPU from nvidia-smi 
print("Using:", device)

Using: cuda


In [14]:
SRC_TEXT.build_vocab(train_dataset)
TGT_TEXT.build_vocab(train_dataset)


sample = dataset[0].src
for tok, id in zip(sample, SRC_TEXT.numericalize([sample])):
    print("{} -> {}".format(tok, id.numpy()[0]))

if -> 16
self -> 10
. -> 4
_ -> 5
cull -> 789
_ -> 5
frequency -> 1261
equals -> 101
to -> 17
integer -> 61
0 -> 100
. -> 4


In [15]:
batch_size = 32

train_iterator, valid_iterator = BucketIterator.splits(
    (train_dataset, val_dataset),
    batch_size = batch_size,
    sort_key = lambda x: len(x.src)+len(x.tgt),
    device = device)

# The iterator generates batches with padded length for sequences with similar sizes, a batch is [seq_length, batch_size]

for i, batch in enumerate(train_iterator):
    print([SRC_TEXT.vocab.itos[id] for id in batch.src.cpu().numpy()[:,4]])
    break

['<sos>', 'call', 'the', 'the', 'method', '_', '_', 'init', '_', '_', 'from', 'the', 'base', 'class', 'of', 'the', 'class', 'Context', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


## Building the model


Sample transformer without positional encoding, it uses the built in transformer model

In [16]:
rand_transformer_model = nn.Transformer() # uses default hyperparameters
src = torch.rand((10, 32, 512)) # [src_seq_length, batch_size, embedding_size]
tgt = torch.rand((20, 32, 512)) # [tgt_seq_length, batch_size, embedding_size]
rand_transformer_model(src, tgt).shape # [tgt_seq_length, batch_size, embedding_size]

torch.Size([20, 32, 512])

In [17]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [18]:
class TransformerModel(nn.Module):

    def __init__(self, src_vocab_size, tgt_vocab_size, embedding_size=128, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        
        self.embedding_size = embedding_size
        self.pos_encoder = PositionalEncoding(embedding_size, dropout)
        self.src_encoder = nn.Embedding(src_vocab_size, embedding_size)
        self.tgt_encoder = nn.Embedding(tgt_vocab_size, embedding_size)
        
        self.transformer = nn.Transformer(d_model=embedding_size, nhead=8, num_encoder_layers=4, num_decoder_layers=4, dim_feedforward=1024)
        self.decoder = nn.Linear(embedding_size, tgt_vocab_size)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.src_encoder.weight.data.uniform_(-initrange, initrange)
        self.tgt_encoder.weight.data.uniform_(-initrange, initrange)
        
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, tgt):
        self.tgt_mask = self._generate_square_subsequent_mask(len(tgt)).to(device)

        src = self.src_encoder(src) * math.sqrt(self.embedding_size)
        src = self.pos_encoder(src)
        
        tgt = self.tgt_encoder(tgt) * math.sqrt(self.embedding_size)
        tgt = self.pos_encoder(tgt)
        
        output = self.transformer(src, tgt, tgt_mask=self.tgt_mask)
        output = self.decoder(output)
        return output

In [19]:
src_vocab_size = len(SRC_TEXT.vocab.stoi)
tgt_vocab_size = len(TGT_TEXT.vocab.stoi)

model = TransformerModel(src_vocab_size, tgt_vocab_size).to(device) 
src = torch.randint(0, src_vocab_size, (10,8), device=device) # [src_seq_length, batch_size]
tgt = torch.randint(0, tgt_vocab_size, (20,8), device=device) # [src_seq_length, batch_size]
model(src, tgt).shape 

torch.Size([20, 8, 3897])

In [20]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    tgt_vocab_size = len(TGT_TEXT.vocab.stoi)
    for i, batch in enumerate(train_iterator):
        encoder_input = batch.src
        decoder_input = batch.tgt[:-1]
        targets = batch.tgt[1:]
        
        optimizer.zero_grad()
        output = model(encoder_input, decoder_input)
        loss = criterion(output.view(-1, tgt_vocab_size), targets.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        total_loss += loss.item()
        log_interval = 200
        if i % log_interval == 0 and i > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, i, len(train_iterator), scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [21]:
def nltk_bleu(refrence, prediction):
    """
    Implementation from ReCode
    and moses multi belu script sets BLEU to 0.0 if len(toks) < 4
    """
    ngram_weights = [0.25] * min(4, len(refrence))
    return sentence_bleu([refrence], prediction, weights=ngram_weights, 
                          smoothing_function=SmoothingFunction().method3)

nltk_bleu([1,2,3,4,5,6], [1,2,5,6])

0.2740311596835683

In [24]:
def evaluate(eval_model, valid_iterator):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    tgt_vocab_size = len(TGT_TEXT.vocab.stoi)
    with torch.no_grad():
        results = []
        for j, batch in enumerate(valid_iterator):
            encoder_input = batch.src
            sos_id = TGT_TEXT.vocab.stoi["<sos>"]
            decoder_input = torch.zeros((1, batch.tgt.shape[1]), dtype=torch.long, device=device).fill_(sos_id)
            targets = batch.tgt[1:]
            
#             for i in range(max_seq_length):
            for i in range(5):
                output = eval_model(encoder_input, decoder_input)
                last_pred = output[-1:].argmax(dim=2)
                # top k sampling
                # indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
                # F.softmax()
                decoder_input = torch.cat((decoder_input, last_pred))
            
            if ((j+1) % 40 == 0):
                print("Done {}/{}".format(j, len(valid_iterator)))
            
            for i in range(decoder_input.shape[1]):
                ref_ids = [id for id in batch.tgt.transpose(0,1)[i] if id not in [0,1,2,3]]
                pred_ids = [id for id in decoder_input.transpose(0,1)[i] if id not in [0,1,2,3]]
                src_ids = [id for id in batch.src.transpose(0,1)[i] if id not in [0,1,2,3]]
                
                src_sent = " ".join([SRC_TEXT.vocab.itos[id] for id in src_ids])
                predicted_sent = " ".join([TGT_TEXT.vocab.itos[id] for id in pred_ids])
                ref_sent = " ".join([TGT_TEXT.vocab.itos[id] for id in ref_ids])
                
                result = {
                            "ref_ids": ref_ids,
                            "pred_ids": pred_ids,
                            "BLEU": nltk_bleu(ref_ids, pred_ids),
                    
                            "src_sent": src_sent, 
                            "ref_sent": ref_sent, 
                            "pred_sent": predicted_sent
                         }
                results.append(result)
        
        overall_BLEU = np.mean([r["BLEU"] for r in results])
        return results, overall_BLEU

Moses Multi-BLEU perl script returns 0.0 for any sentence less than 4 tokens long.
It will be best to use a function by NLTK

In [116]:
get_moses_multi_bleu(["this is a test"], ["this is a for"])

0.0

In [117]:
TGT_TEXT.vocab.itos[:10]

['<unk>', '<pad>', '<sos>', '<eos>', '_', '.', '(', ')', ',', "'"]

In [25]:
results, BLEU = evaluate(model, valid_iterator)

Done 39/177
Done 79/177
Done 119/177
Done 159/177


In [26]:
for r in results[:100:30]:
    print("Query\t\t: {}\nTrue Translation: {}\nPrediction\t: {}\n\n".format(r["src_sent"], r["ref_sent"], r["pred_sent"]))

print("Overall BLEU:",BLEU)

Query		: try ,
True Translation: try :
Prediction	: 


Query		: return parser
True Translation: return parser
Prediction	: 


Query		: try ,
True Translation: try :
Prediction	: 


Query		: try ,
True Translation: try :
Prediction	: 


Overall BLEU: 0.0


In [120]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

In [121]:
logits = torch.tensor([0.1,0.0,0.9])
top_k = top_k_top_p_filtering(logits, top_k=2)
probabilities = F.softmax(top_k, dim=-1)
torch.multinomial(probabilities, 1)

tensor([2])

In [22]:
best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    print("######## Finished Epoch {} #########".format(epoch))
#     val_loss = evaluate(model, val_data)
#     print('-' * 89)
#     print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
#           'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
#                                      val_loss, math.exp(val_loss)))
#     print('-' * 89)

#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         best_model = model

    scheduler.step()

| epoch   1 |   200/  411 batches | lr 5.00 | ms/batch 32.68 | loss  3.68 | ppl    39.59
| epoch   1 |   400/  411 batches | lr 5.00 | ms/batch 33.66 | loss  2.89 | ppl    17.95
######## Finished Epoch 1 #########
| epoch   2 |   200/  411 batches | lr 4.75 | ms/batch 33.01 | loss  2.52 | ppl    12.40
| epoch   2 |   400/  411 batches | lr 4.75 | ms/batch 32.54 | loss  2.39 | ppl    10.86
######## Finished Epoch 2 #########
| epoch   3 |   200/  411 batches | lr 4.51 | ms/batch 32.17 | loss  2.28 | ppl     9.75
| epoch   3 |   400/  411 batches | lr 4.51 | ms/batch 32.60 | loss  2.32 | ppl    10.17
######## Finished Epoch 3 #########
| epoch   4 |   200/  411 batches | lr 4.29 | ms/batch 32.64 | loss  2.24 | ppl     9.42
| epoch   4 |   400/  411 batches | lr 4.29 | ms/batch 32.17 | loss  2.20 | ppl     9.04
######## Finished Epoch 4 #########
| epoch   5 |   200/  411 batches | lr 4.07 | ms/batch 32.48 | loss  2.16 | ppl     8.70
| epoch   5 |   400/  411 batches | lr 4.07 | ms/batch 

KeyboardInterrupt: 